In [2]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: keras
    Found existing installation: keras 3.0.5
    Uninstalling keras-3.0.5:
      Successfully uninstalled keras-3.0.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.


In [4]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Filter out CUDA-related warnings
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

# Function to load images and labels with error handling
def load_images_and_labels(image_dirs, img_size=(160, 160)):
    images, labels = [], []
    for label, image_dir in enumerate(image_dirs):
        for filename in os.listdir(image_dir):
            if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue
            img_path = os.path.join(image_dir, filename)
            try:
                img = load_img(img_path, target_size=img_size)
                images.append(img_to_array(img) / 255.0)
                labels.append(label)
            except IOError as e:
                print(f"Error loading image {img_path}: {e}")
    return np.array(images), np.array(labels)

# Directories for real and fake images
real_images_dir = '/kaggle/input/casia-dataset/CASIA2/Au'
fake_images_dir = '/kaggle/input/casia-dataset/CASIA2/Tp'

# Load dataset and split
images, labels = load_images_and_labels([real_images_dir, fake_images_dir])
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

# Model definition
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_images, train_labels, epochs=25, validation_data=(test_images, test_labels), verbose=1)

# Optional model evaluation, uncomment to use
#loss, accuracy = model.evaluate(test_images, test_labels)
# print(f"Test accuracy: {accuracy*100:.2f}%")


Epoch 1/25
238/238 [==============================] - 284s 1s/step - loss: 0.5478 - accuracy: 0.7797 - val_loss: 0.5100 - val_accuracy: 0.7833
Epoch 2/25
238/238 [==============================] - 251s 1s/step - loss: 0.4986 - accuracy: 0.7826 - val_loss: 0.4916 - val_accuracy: 0.7833
Epoch 3/25
238/238 [==============================] - 281s 1s/step - loss: 0.4707 - accuracy: 0.7855 - val_loss: 0.4920 - val_accuracy: 0.7833
Epoch 4/25
238/238 [==============================] - 250s 1s/step - loss: 0.4311 - accuracy: 0.7996 - val_loss: 0.4854 - val_accuracy: 0.7817
Epoch 5/25
238/238 [==============================] - 279s 1s/step - loss: 0.3839 - accuracy: 0.8222 - val_loss: 0.4872 - val_accuracy: 0.7870
Epoch 6/25
238/238 [==============================] - 252s 1s/step - loss: 0.3385 - accuracy: 0.8403 - val_loss: 0.5108 - val_accuracy: 0.7922
Epoch 7/25
  9/238 [>.............................] - ETA: 3:40 - loss: 0.3217 - accuracy: 0.8542

KeyboardInterrupt: 

In [9]:
# Function to predict the class of an image with preprocessing consistency
def predict_image_class(image_path, model, img_size=(160, 160)):
    img = load_img(image_path, target_size=img_size)
    img_array = np.expand_dims(img_to_array(img) / 255.0, axis=0)
    prediction = model.predict(img_array)
    return 'Real' if prediction[0] < 0.5 else 'Fake'

# Corrected image path for prediction
image_path = '//kaggle/input/casia-dataset/CASIA2/Au/Au_ani_00002.jpg'
print(f"The image is predicted to be: {predict_image_class(image_path, model)}")


1/1 [==============================] - 0s 34ms/step
The image is predicted to be: Real
